In [1]:
import numpy as np
import pandas as pd
import sys
import os
from codebase.file_utils import save_obj, load_obj
from codebase.model_fit_cont import energy_score
import altair as alt
alt.data_transformers.disable_max_rows()

%load_ext autoreload
%autoreload 2

## Continous data

In [2]:
def variogram_score(y_pred, y_obs, p = 0.5):
    mcmc_lenght = y_pred.shape[0]
    y_obs_length = y_obs.shape[0]
    p = 0.5

    s = 0.
    for i in range(y_obs_length):
        for j in range(y_obs_length):
            s1 = np.abs(y_obs[i] - y_obs[j])**p
            s2 = (-1./mcmc_lenght) * np.array([np.abs(X[k,i] - X[k,j])**p for k in range(mcmc_lenght)]).sum()
            s_ij = (s1 + s2)**2
            s = s + s_ij
    return s


In [3]:
y_obs = np.arange(4); print(y_obs)
X = np.arange(1,13).reshape((3,4)); print(X)
print(energy_score(X, y_obs))
print(variogram_score(X, y_obs))


[0 1 2 3]
[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
6.444444444444445
0.0


In [4]:
y_obs = np.array([1,1,2]); print(y_obs)
X = np.arange(1,31).reshape((10,3)); print(X)
print(energy_score(X, y_obs))
print(variogram_score(X, y_obs))


[1 1 2]
[[ 1  2  3]
 [ 4  5  6]
 [ 7  8  9]
 [10 11 12]
 [13 14 15]
 [16 17 18]
 [19 20 21]
 [22 23 24]
 [25 26 27]
 [28 29 30]]
16.005927778284295
2.3431457505076203


## Binary Data

In [19]:
from numpy.linalg import norm

def crps_score(y_pred, y_obs):
    # y_obs has dim J (1 observation)
    # y_pred has dim m x J (m posterior samnples)
    m = y_pred.shape[0]
    # sum |Xi - y| for all i
    s1 = np.abs(y_pred - y_obs).sum()
    # sum all pairs |Xi - Xj|| for all i and all j
    s2 = 0.
    for i  in range(m):
        for j in range(m):
            s2 += np.abs(y_pred[i] - y_pred[j])
    s2
    
    # remove the 1/2 from the second sum because we summed each distinct pair only once
    return (s1/m) - (s2/(m**2))

In [20]:
y_obs = np.array([1,0,0]); print(y_obs)
X = np.array([0.6, 0.5 ,0.4]); print(X)
crps_score(X, y_obs)

[1 0 0]
[0.6 0.5 0.4]


0.3444444444444445

In [21]:
y_obs = np.array([0,0,1]); print(y_obs)
X = np.array([0.99, 0.99, 0.01]); print(X)
crps_score(X, y_obs)

[0 0 1]
[0.99 0.99 0.01]


0.5544444444444443

In [9]:
from scipy.spatial.distance import pdist
ss = 0
for i  in range(3):
    for j in range(3):
        ss += np.abs(X[i] - X[j])
ss

0.7999999999999998